In [3]:
import sqlalchemy
import requests
import pandas as pd
import pymysql

# 과제

stock.to_sql(name='daebak_jongmok', con=engine, if_exists='append', index=False)

대상 : 2585 종목

table 명 : deabak_jongmok

컬럼 : code, code_name, date, close, before, open, high, low, trade, ma05, ma20, ma60, ma120

기간 : 네이버 수집시 page 100페이지

주의사항 : NA 삭제, 중복 허용 X

ISU_CD	ISU_SRT_CD	ISU_NM	ISU_ABBRV	ISU_ENG_NM	LIST_DD	MKT_TP_NM	SECUGRP_NM	SECT_TP_NM	KIND_STKCERT_TP_NM	PARVAL	LIST_SHRS


date	close	before	open	high	low	trade	요일

# 주식 데이터

In [6]:
url_post = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
head = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36"}
payload = {'bld': 'dbms/MDC/STAT/standard/MDCSTAT01901',
'mktId': 'ALL',
'share': '1',
'csvxls_isNo': 'false'}

r2 = requests.post(url_post, headers=head, data=payload)

r2.json()['OutBlock_1'] #딕셔너리는 데이터프레임으로 만들 수 있음

master = pd.DataFrame(r2.json()['OutBlock_1'])

# 최종 코드

In [8]:
stock_url = "https://finance.naver.com/item/sise_day.naver?code={}&page={}"
head = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36"}
total = []

for i, code in enumerate(master.values[:100]):
    if i % 10 == 0:
        print(f'{i}번째 데이터를 불러오고 있습니다...')
    try:
        nows = []
        for x in range(1, 13):
            r = requests.get(stock_url.format(code[1], x), headers=head)
            now_code = pd.DataFrame(code.reshape(1, 12), columns=['~', 'code', 'code_name', '~', '~', 'date', '~', '~', '~' ,'~', '~', '~'])[['code' ,'code_name']]
            now = pd.read_html(r.text)[0].dropna()
            now['code'] = now_code.code.to_list()[0]
            now['code_name'] = now_code.code_name.to_list()[0]
            nows.append(now)
        df1 = pd.concat(nows)
        df1 = df1.sort_values(by='날짜')
        df1['ma05'] = df1.종가.rolling(5).mean()
        df1['ma20'] = df1.종가.rolling(20).mean()
        df1['ma60'] = df1.종가.rolling(60).mean()
        df1['ma120'] = df1.종가.rolling(120).mean()
        total.append(df1.iloc[-1:])
    except:
        print('error')
daebak = pd.concat(total)
daebak.columns = ['date', 'close', 'before', 'open', 'high', 'low', 'trade', 'code', 'code_name', 'ma05', 'ma20', 'ma60', 'ma120']
daebak

0번째 데이터를 불러오고 있습니다...
10번째 데이터를 불러오고 있습니다...
20번째 데이터를 불러오고 있습니다...
30번째 데이터를 불러오고 있습니다...
40번째 데이터를 불러오고 있습니다...
50번째 데이터를 불러오고 있습니다...
60번째 데이터를 불러오고 있습니다...
70번째 데이터를 불러오고 있습니다...
80번째 데이터를 불러오고 있습니다...
90번째 데이터를 불러오고 있습니다...


,date,close,before,open,high,low,trade,code,code_name,ma05,ma20,ma60,ma120
1,2021.10.08,7550.0,230.0,7900.0,7970.0,7480.0,35594.0,098120,(주)마이크로컨텍솔루션,7590.0,8479.00,8695.333333,7729.416667
1,2021.10.08,877.0,13.0,902.0,902.0,850.0,221995.0,131100,(주)스카이이앤엠,887.8,956.95,1198.283333,1404.100000
1,2021.10.08,7170.0,30.0,7210.0,7360.0,7160.0,259913.0,009520,(주)포스코엠텍,7342.0,7878.00,8315.833333,8715.916667
1,2021.10.08,5690.0,20.0,5790.0,5790.0,5630.0,124442.0,095570,AJ네트웍스보통주,5666.0,5898.50,5999.333333,5864.208333
1,2021.10.08,26300.0,100.0,26350.0,26850.0,25950.0,13340.0,006840,AK홀딩스보통주,26610.0,27087.50,27355.833333,29984.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,2021.10.08,4590.0,145.0,4455.0,4650.0,4420.0,48278.0,123700,SJM보통주,4413.0,4655.75,4964.583333,5248.791667
1,2021.10.08,3520.0,5.0,3630.0,3630.0,3515.0,24422.0,025530,SJM홀딩스보통주,3524.0,3809.50,4194.583333,4422.000000
1,2021.10.08,230500.0,500.0,235000.0,235000.0,228500.0,1461.0,03473K,SK1우선주,232200.0,241375.00,244833.333333,250233.333333
1,2021.10.08,157500.0,3000.0,158000.0,162000.0,155500.0,529266.0,011790,SKC보통주,159400.0,171450.00,163333.333333,152270.833333


In [229]:
daebak.iloc[1,:]

date          2021.10.08
close              877.0
before              13.0
open               902.0
high               902.0
low                850.0
trade           221995.0
code              131100
code_name      (주)스카이이앤엠
ma05               887.8
ma20              956.95
ma60         1198.283333
ma120             1404.1
Name: 1, dtype: object

In [237]:
import pymysql
try:
    con = pymysql.connect(host='3.34.134.218', user='root', port=3306, password='1234',  
                          db='daebak_jongmok', charset='utf8')
    cur = con.cursor()
except Exception as e:
    print (e)   

In [238]:
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + "1234" + "@3.34.134.218/stocks", encoding='utf8')
stock.to_sql(name='daebak_jongmok', con=engine, if_exists='append', index=False)

UnicodeEncodeError: 'charmap' codec can't encode characters in position 76-77: character maps to <undefined>

In [244]:
master

,ISU_CD,ISU_SRT_CD,ISU_NM,ISU_ABBRV,ISU_ENG_NM,LIST_DD,MKT_TP_NM,SECUGRP_NM,SECT_TP_NM,KIND_STKCERT_TP_NM,PARVAL,LIST_SHRS
0,KR7098120009,098120,(주)마이크로컨텍솔루션,마이크로컨텍솔,"Micro Contact Solution Co.,Ltd.",2008/09/23,KOSDAQ,주권,벤처기업부,보통주,500,"8,312,766"
1,KR7131100000,131100,(주)스카이이앤엠,스카이이앤엠,"SKY E&M Co., Ltd.",2011/06/14,KOSDAQ,주권,관리종목(소속부없음),보통주,100,"53,703,566"
2,KR7009520008,009520,(주)포스코엠텍,포스코엠텍,"POSCO M-TECH CO.,LTD.",1997/11/10,KOSDAQ,주권,우량기업부,보통주,500,"41,642,703"
3,KR7095570008,095570,AJ네트웍스보통주,AJ네트웍스,"AJ Networks Co.,Ltd.",2015/08/21,KOSPI,주권,,보통주,"1,000","46,822,295"
4,KR7006840003,006840,AK홀딩스보통주,AK홀딩스,"AK Holdings, Inc.",1999/08/11,KOSPI,주권,,보통주,"5,000","13,247,561"
...,...,...,...,...,...,...,...,...,...,...,...,...
2580,KR7000542001,000547,흥국화재해상보험2우선주(신형),흥국화재2우B,HeungkukFire&MarineIns(2PB),1999/08/09,KOSPI,주권,,신형우선주,"5,000","153,600"
2581,KR7000540005,000540,흥국화재해상보험보통주,흥국화재,HeungkukFire&MarineInsurance,1974/12/05,KOSPI,주권,,보통주,"5,000","64,242,645"
2582,KR7003280005,003280,흥아해운보통주,흥아해운,HeungaShipping,1976/06/29,KOSPI,주권,,보통주,500,"239,125,028"
2583,KR7037440005,037440,희림종합건축사사무소,희림,Heerim Architects & Planners,2000/02/03,KOSDAQ,주권,중견기업부,보통주,500,"13,922,475"


# 시도 및 검증

In [193]:
df1 = pd.concat(total)
df1 = df1[df1['code_name'] == '(주)마이크로컨텍솔루션']
df1 = df1.sort_values(by='날짜')

df1['ma05'] = df1.종가.rolling(5).mean()
df1['ma20'] = df1.종가.rolling(20).mean()
df1['ma60'] = df1.종가.rolling(60).mean()
df1['ma120'] = df1.종가.rolling(120).mean()
df1.iloc[-1:]

,날짜,종가,전일비,시가,고가,저가,거래량,code,code_name,ma05,ma20,ma60,ma120
1,2021.10.08,7550.0,230.0,7900.0,7970.0,7480.0,35594.0,098120,(주)마이크로컨텍솔루션,7590.0,8479.0,8695.333333,7729.416667


In [178]:
datetime.now()

datetime.datetime(2021, 10, 11, 1, 39, 36, 790647)

In [167]:
df1.종가.rolling(5).mean()

1        NaN
2        NaN
3        NaN
4        NaN
5     7590.0
       ...  
9     6116.0
10    6162.0
11    6198.0
12    6192.0
13    6170.0
Name: 종가, Length: 120, dtype: float64

In [151]:
now.columns = ['code', 'code_name', 'date', 'close', 'before', 'open', 'high', 'low', 'trade', 'ma05', 'ma20', 'ma60', 'ma120']

ValueError: Length mismatch: Expected axis has 9 elements, new values have 13 elements

In [109]:
now

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.04.28,1795.0,10.0,1810.0,1825.0,1770.0,713592.0
2,2021.04.27,1805.0,0.0,1825.0,1825.0,1765.0,570676.0
3,2021.04.26,1805.0,70.0,1740.0,1815.0,1705.0,1007158.0
4,2021.04.23,1735.0,10.0,1745.0,1770.0,1710.0,568109.0
5,2021.04.22,1745.0,20.0,1720.0,1775.0,1690.0,828614.0
9,2021.04.21,1725.0,55.0,1765.0,1765.0,1715.0,577617.0
10,2021.04.20,1780.0,45.0,1810.0,1810.0,1745.0,1021765.0
11,2021.04.19,1825.0,35.0,1850.0,1850.0,1805.0,603925.0
12,2021.04.16,1860.0,5.0,1865.0,1870.0,1805.0,611787.0
13,2021.04.15,1865.0,0.0,1835.0,1870.0,1805.0,1026185.0


In [112]:
now_code

,code,code_name
0,131100,(주)스카이이앤엠


In [144]:
now_code.code.to_list()

['131100']

In [145]:
now['code'] = now_code.code.to_list()[0]
now['code_name'] = now_code.code_name.to_list()[0]

In [148]:
now

,날짜,종가,전일비,시가,고가,저가,거래량,code,code_name
1,2021.04.28,1795.0,10.0,1810.0,1825.0,1770.0,713592.0,131100,(주)스카이이앤엠
2,2021.04.27,1805.0,0.0,1825.0,1825.0,1765.0,570676.0,131100,(주)스카이이앤엠
3,2021.04.26,1805.0,70.0,1740.0,1815.0,1705.0,1007158.0,131100,(주)스카이이앤엠
4,2021.04.23,1735.0,10.0,1745.0,1770.0,1710.0,568109.0,131100,(주)스카이이앤엠
5,2021.04.22,1745.0,20.0,1720.0,1775.0,1690.0,828614.0,131100,(주)스카이이앤엠
9,2021.04.21,1725.0,55.0,1765.0,1765.0,1715.0,577617.0,131100,(주)스카이이앤엠
10,2021.04.20,1780.0,45.0,1810.0,1810.0,1745.0,1021765.0,131100,(주)스카이이앤엠
11,2021.04.19,1825.0,35.0,1850.0,1850.0,1805.0,603925.0,131100,(주)스카이이앤엠
12,2021.04.16,1860.0,5.0,1865.0,1870.0,1805.0,611787.0,131100,(주)스카이이앤엠
13,2021.04.15,1865.0,0.0,1835.0,1870.0,1805.0,1026185.0,131100,(주)스카이이앤엠


In [138]:
now_code.code.values()

TypeError: 'numpy.ndarray' object is not callable

In [90]:
pd.DataFrame(code.reshape(1, 12), columns=['~', 'code', 'code_name', '~', '~', 'date', '~', '~', '~' ,'~', '~', '~'])[['code' ,'code_name', 'date']]

,code,code_name,date
0,131100,(주)스카이이앤엠,2011/06/14


In [88]:
code.reshape(1, 12)

array([['KR7131100000', '131100', '(주)스카이이앤엠', '스카이이앤엠',
        'SKY E&M Co., Ltd.', '2011/06/14', 'KOSDAQ', '주권', '관리종목(소속부없음)',
        '보통주', '100', '53,703,566']], dtype=object)

In [85]:
code

array(['KR7131100000', '131100', '(주)스카이이앤엠', '스카이이앤엠',
       'SKY E&M Co., Ltd.', '2011/06/14', 'KOSDAQ', '주권', '관리종목(소속부없음)',
       '보통주', '100', '53,703,566'], dtype=object)

In [77]:
pd.merge(now, code, how='outter',on='날짜')

TypeError: Can only merge Series or DataFrame objects, a <class 'numpy.ndarray'> was passed

In [74]:
code

array(['KR7131100000', '131100', '(주)스카이이앤엠', '스카이이앤엠',
       'SKY E&M Co., Ltd.', '2011/06/14', 'KOSDAQ', '주권', '관리종목(소속부없음)',
       '보통주', '100', '53,703,566'], dtype=object)

In [69]:
total

[            날짜      종가    전일비      시가      고가      저가      거래량
 1   2021.10.08  7550.0  230.0  7900.0  7970.0  7480.0  35594.0
 2   2021.10.07  7780.0  530.0  7290.0  7780.0  7250.0  90165.0
 3   2021.10.06  7250.0  320.0  7570.0  7700.0  7170.0  70338.0
 4   2021.10.05  7570.0  230.0  7600.0  7880.0  7410.0  75793.0
 5   2021.10.01  7800.0  270.0  8070.0  8140.0  7700.0  63533.0
 9   2021.09.30  8070.0   20.0  8080.0  8120.0  7930.0  36865.0
 10  2021.09.29  8090.0   60.0  7980.0  8210.0  7830.0  43255.0
 11  2021.09.28  8030.0  190.0  8180.0  8670.0  8000.0  52302.0
 12  2021.09.27  8220.0  230.0  8450.0  8520.0  8190.0  46535.0
 13  2021.09.24  8450.0  170.0  8580.0  8800.0  8400.0  50402.0,
             날짜      종가    전일비       시가       고가      저가       거래량
 1   2021.09.23  8620.0  360.0   8860.0   8890.0  8300.0   63688.0
 2   2021.09.17  8980.0  200.0   8730.0   8980.0  8700.0   31531.0
 3   2021.09.16  8780.0  210.0   8990.0   9080.0  8710.0   53403.0
 4   2021.09.15  8990.0   1

In [65]:
jongmok = pd.concat(total)

In [256]:
from datetime import datetime

In [285]:
day = datetime.today().strftime("%Y.%m.%d")
day

'2021.10.08'

In [293]:
jongmok['날짜']

1     2021.10.08
2     2021.10.07
3     2021.10.06
4     2021.10.05
5     2021.10.01
         ...    
9     2021.09.30
10    2021.09.29
11    2021.09.28
12    2021.09.27
13    2021.09.24
Name: 날짜, Length: 25804, dtype: object

In [ ]:
jongmok = jongmok[jongmok['날짜'] == '2021.10.08']
jongmok.head()

In [ ]:
master

In [ ]:
merge_outer = pd.merge(jongmok,master, how='inner',on='id')

In [ ]:
master.iloc[:0,:].to_sql(name='day_stock', con=engine, if_exists='append', index=False)

# DB 접속 
try:
    con = pymysql.connect(host='3.34.134.218', user='root', port=3306, password='1234',  
                          db='daebak_jongmok', charset='utf8')
    cur = con.cursor()
except Exception as e:
    print (e)   
sql = 'INSERT INTO daebak_jongmok_stock VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'

cur.execute('desc daebak_jongmok')
#con.commit()